In [4]:
"""
importing examples 
├── project (package)
│   ├── src (package)
│   │   ├── __init__.py
│   │   └── utils.py
│   |── clustering.ipynb
    └── __init__.py

import src ---> src.utils.test(10)
from src import utils ---> utils.test(10)
from src.utils import test ---> test(10) 
----------------------------------------
please import each library in its own line 
e.g.

((THIS IS TRUE))
import os
import sys

((THIS IS WRONG))
import os,sys

----------------------------------------
in the package case its ok to import all things in one liune
e.g. 
from src import utils,constants   ((Thats OK !!))
----------------------------------------
import proiority : (very important)
1) Standard library imprts  e.g. sys,os
2) Third party imports e.g. pandas , numpy
3) Local imports e.g. utils , constants and other things inside the project
"""


'\nimporting examples \n├── project (package)\n│   ├── src (package)\n│   │   ├── __init__.py\n│   │   └── utils.py\n│   |── clustering.ipynb\n    └── __init__.py\n\nimport src ---> src.utils.test(10)\nfrom src import utils ---> utils.test(10)\nfrom src.utils import test ---> test(10) \n----------------------------------------\nplease import each library in its own line \ne.g.\n\n((THIS IS TRUE))\nimport os\nimport sys\n\n((THIS IS WRONG))\nimport os,sys\n\n----------------------------------------\nin the package case its ok to import all things in one liune\ne.g. \nfrom src import utils,constants   ((Thats OK !!))\n----------------------------------------\nimport proiority : (very important)\n1) Standard library imprts  e.g. sys,os\n2) Third party imports e.g. pandas , numpy\n3) Local imports e.g. utils , constants and other things inside the project\n'

In [5]:
# Standard libraries Imports
import json
# Third party Imports
import pandas as pd
import numpy as np
import requests
# Local imports
from src.constants import DATA_API

In [6]:
response = requests.get(DATA_API) # get the data from the API
json_response = json.loads(response.content) # convert the response from binary format to json (dict)
data =pd.DataFrame(json_response) # create the dataframe

In [7]:
data.head()

,hh_id,weight,cms_zone,survey_mode,person_id,person_num,is_participant,first_travel_date,last_travel_date,smartphone_type,...,num_complete_weekend_days,num_complete_weekdays,made_trips,work_cms_zone,school_cms_zone,school_park_amount_day,work_park_amount_week,school_park_amount_week,work_park_amount_month,work_park_amount_day
0,191546322,0.00000,Inner Brooklyn,1,19154632203,3,0,2019-06-24T00:00:00.000,2019-06-30T00:00:00.000,995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,191546322,0.00000,Inner Brooklyn,1,19154632202,2,0,2019-06-24T00:00:00.000,2019-06-30T00:00:00.000,995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,191546322,212.12729,Inner Brooklyn,1,19154632201,1,1,2019-06-24T00:00:00.000,2019-06-30T00:00:00.000,1,...,2,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19241657,0.00000,Middle Queens,1,1924165703,3,0,2019-05-29T00:00:00.000,2019-06-04T00:00:00.000,995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19241657,0.00000,Middle Queens,1,1924165702,2,0,2019-05-29T00:00:00.000,2019-06-04T00:00:00.000,995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
